In [1]:
import sys
sys.path.append('..')

In [2]:
import itertools

import numpy as np
import pandas as pd
import tensorflow as tf
# tf.enable_eager_execution()

import few_shot.experiments.fashion.config as config
from few_shot.experiments.fashion import evaluate_fashion_few_shot
from few_shot.dataset.fashion import fashion_dfs
from few_shot.dataset.image_pipeline import augmented_img_pipeline_fn, class_augmentation_fn

W0622 09:15:15.492971 139958080591680 deprecation_wrapper.py:119] From ../few_shot/dataset/__init__.py:65: The name tf.data.Iterator is deprecated. Please use tf.compat.v1.data.Iterator instead.



In [3]:
np.random.seed(23)
tf.random.set_random_seed(29)
train_df, val_df, test_df = fashion_dfs(dataset_path='../datasets/fashion-dataset')

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

107


In [4]:
n_shots = 5
k_way_test = 15

The keras ResNet50 model requires images to be preprocessed in a particular way.

In [5]:
def preprocess_class_augm(k):
    augmented_batch_fn = class_augmentation_fn(k)
    
    def preprocess_imagenet(input_tuple, qy):
        sx, sy, qx = input_tuple
        
        return (
            (
                tf.keras.applications.resnet50.preprocess_input(sx),
                sy,
                tf.keras.applications.resnet50.preprocess_input(qx)
            ),
            qy
        )
    
    return preprocess_imagenet

In [6]:
img_shape = config.IMG_SHAPE
emb_input = tf.keras.layers.Input(shape=img_shape)

resnet_model = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    input_shape=img_shape,
    pooling='avg'
)

# start by only training the last embedding block
for layer in resnet_model.layers[:-32]:  # 11 for conv block, 10 * 2 for the identity blocks, 1 for pooling layer
    layer.trainable = False
    
for layer in resnet_model.layers[-32:]:
    layer.trainable = True

In [7]:
resnet = resnet_model(emb_input)

# protonet expects a dense layer as its input
flattened_resnet = tf.keras.layers.Flatten()(resnet)

embedding_model = tf.keras.models.Model(inputs=emb_input, outputs=flattened_resnet)

In [8]:
print(f'Running fashion experiment {n_shots}-shot, {k_way_test} way')
assert k_way_test <= test_df.class_name.nunique()
result = evaluate_fashion_few_shot(train_df=train_df,
                                   val_df=val_df,
                                   test_df=test_df,
                                   n_shot=n_shots,
                                   k_way_test=k_way_test,
                                   patience=5,
                                   restore_best_weights=True,
                                   augment=True,
                                   post_processing_fn=preprocess_class_augm(config.K_WAY_TRAIN),
                                   embedding_fn=lambda x: embedding_model
                                  
                                  )

W0622 09:15:25.372928 139958080591680 deprecation.py:323] From /home/andre/Projects/fashion_few_shot/.venv/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
W0622 09:15:25.414747 139958080591680 depr

Running fashion experiment 5-shot, 15 way
{'augment': True, 'post_processing_fn': <function preprocess_class_augm.<locals>.preprocess_imagenet at 0x7f49f3e7dd08>, 'validation_metric': 'loss', 'reduction_factor': 0.75, 'reduce_lr_on_plateau': False, 'embedding_fn': <function <lambda> at 0x7f49f3e7dae8>, 'restore_best_weights': True, 'patience': 5, 'img_shape': (80, 60, 3), 'test_eps': 1000, 'n_epochs': 100, 'k_way_train': 30, 'n_queries_test': 1, 'n_queries_train': 5, 'lr': 0.001, 'k_way_test': 15, 'n_shot': 5, 'eps_per_epoch': 100}


W0622 09:15:25.746461 139958080591680 deprecation.py:323] From ../few_shot/dataset/__init__.py:101: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
W0622 09:15:35.592292 139958080591680 deprecation_wrapper.py:119] From ../few_shot/model.py:69: The name tf.linalg.transpose is deprecated. Please use tf.linalg.matrix_transpose instead.

W0622 09:15:35.595493 139958080591680 deprecation_wrapper.py:119] From ../few_shot/model.py:70: The name tf.squared_difference is deprecated. Please use tf.math.squared_difference instead.



Training:

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
100/100 [==============================] - 334s 3s/step - loss: 7.4726 - categorical_accuracy: 0.6955
{'loss': [7.472556512951851], 'categorical_accuracy': [0.69553334], 'lr': [0.001]}
Validation:
100/100 [==============================] - 137s 1s/step - loss: 1.6613 - categorical_accuracy: 0.5153
epoch 0: val_loss: 1.6612762129306793, val_cat_accuracy: 0.515333354473114
Training:

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 2/2
100/100 [==============================] - 280s 3s/step - loss: 0.4581 - categorical_accuracy: 0.8468
{'loss': [0.45805410817265513], 'categorical_accuracy': [0.8468], 'lr': [0.001]}
Validation:
100/100 [==============================] - 132s 1s/step - loss: 1.6767 - categorical_accuracy: 0.5367
epoch 1: val_loss: 1.6767052817344665, val_cat_accuracy: 0.5366666913032532
Training:

Epoch 00003: LearningRateScheduler reducin